<a href="https://colab.research.google.com/github/sharvariK-11/NST-/blob/main/Chapter2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
import transformers
!pip install transformers[sentencepiece]

     |████████████████████████████████| 4.0 MB 4.2 MB/s 
     |████████████████████████████████| 596 kB 34.9 MB/s 
     |████████████████████████████████| 77 kB 6.2 MB/s 
     |████████████████████████████████| 880 kB 59.3 MB/s 
     |████████████████████████████████| 6.6 MB 41.1 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=d3d20cb8c519808c78a58e1d261841b2c52019885ddd5d0af1c34c82dfebe6ef
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 3.5 MB/s 


    TRANSFORMERS= tokeniser-model-post processing


tokeniser - Splitting the input into words, subwords, or symbols (like punctuation) that are called tokens
Mapping each token to an integer
Adding additional inputs that may be useful to the model

In [ ]:
from transformers import AutoTokenizer

checkpoint= "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer=AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [ ]:
raw_inputs= ['I like this', "I have been waiting for this since long"]

inputs=tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[ 101, 1045, 2066, 2023,  102,    0,    0,    0,    0,    0],
        [ 101, 1045, 2031, 2042, 3403, 2005, 2023, 2144, 2146,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


Model 
downloaded the same checkpoint we used in our pipeline before (it should actually have been cached already) and instantiated a model with it.

In [ ]:
from transformers import AutoModel
checkpoint="distilbert-base-uncased-finetuned-sst-2-english"
model=AutoModel.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing DistilBertModel: ['classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'pre_classifier.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
outputs=model(**inputs)
print(outputs.last_hidden_state.shape)

torch.Size([2, 10, 768])


In [ ]:
from transformers import AutoModelForSequenceClassification

checkpoint="distilbert-base-uncased-finetuned-sst-2-english"
model=AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)
print(outputs.logits.shape)

torch.Size([2, 2])


2 sentences 2 labels

output from model not = probabilities

In [ ]:
print(outputs.logits)

tensor([[-4.3017,  4.6251],
        [ 1.3390, -1.0682]], grad_fn=<AddmmBackward0>)


post processing


In [ ]:
import torch
predictions=torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[1.3277e-04, 9.9987e-01],
        [9.1738e-01, 8.2619e-02]], grad_fn=<SoftmaxBackward0>)


To get the labels corresponding to each position

In [ ]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

###Models-loading,saving models

In [ ]:
from transformers import BertConfig, BertModel

config=BertConfig()
model=BertModel(config)

loading pre trained model

In [ ]:
from transformers import BertModel
model=BertModel.from_pretrained("bert-base-cased")

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model.save_pretrained("directory_on_my_computer")

Tensors only accept rectangular shapes 

model inputs- model-output.
output=model(model_inputs)

### TOKENIZERS - translate text into data that can be processed by the model.


word based

In [ ]:
tokenized_text="I am good at football.".split()
print(tokenized_text)

['I', 'am', 'good', 'at', 'football.']


character based - reduce the amount of unknown tokens - letters, numbers, special characters

sub word tokenization
frequently used words should not be split into smaller subwords, but rare words should be decomposed into meaningful subwords.

### loading and saving tokenizers


In [ ]:
from transformers import BertTokenizer 
tokenizer=BertTokenizer.from_pretrained("bert-base-cased")

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

In [ ]:
tokenizer("I am good at football.")

{'input_ids': [101, 146, 1821, 1363, 1120, 1709, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer.save_pretrained("directory_on_my_computer")

('directory_on_my_computer/tokenizer_config.json',
 'directory_on_my_computer/special_tokens_map.json',
 'directory_on_my_computer/vocab.txt',
 'directory_on_my_computer/added_tokens.json')

### *Encoding*-Translating text to numbers is known as encoding.  two-step process: the tokenization, followed by the conversion to input IDs.

In [ ]:
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained("bert-base-cased")
sequence="let's do tokenisation"
tokens=tokenizer.tokenize(sequence)
print(tokens)

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

['let', "'", 's', 'do', 'token', '##isation']


In [ ]:
ids=tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[1519, 112, 188, 1202, 22559, 5771]


###*Decoding* : from vocabulary indices, we want to get a string.

In [ ]:
decoded_string= tokenizer.decode(ids)
print(decoded_string)

let's do tokenisation


Multiple sequences= Batched sequences 
. To get the same result when passing individual sentences of different lengths through the model or when passing a batch with the same sentences and padding applied, we need to tell those attention layers to ignore the padding tokens. This is done by using an attention mask.


In [ ]:
from transformers import AutoTokenizer

checkpoint= "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer=AutoTokenizer.from_pretrained(checkpoint)
sentences=["I have been waiting for long", "I hate coffee"]
print(tokenizer(sentences))
print(tokenizer(sentences, padding=True))


tokens=[tokenizer.tokenize(sentence) for sentence in sentences]
ids=[tokenizer.convert_tokens_to_ids(token) for token in tokens]
print(ids)

{'input_ids': [[101, 1045, 2031, 2042, 3403, 2005, 2146, 102], [101, 1045, 5223, 4157, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}
{'input_ids': [[101, 1045, 2031, 2042, 3403, 2005, 2146, 102], [101, 1045, 5223, 4157, 102, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 0, 0, 0]]}
[[1045, 2031, 2042, 3403, 2005, 2146], [1045, 5223, 4157]]


In [ ]:
from transformers import AutoTokenizer

tokenizer=AutoTokenizer.from_pretrained(checkpoint)
tokenizer.pad_token_id

0

In [ ]:
from transformers import AutoTokenizer

checkpoint= "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer=AutoTokenizer.from_pretrained(checkpoint)
sentences=["I have been waiting for long", "I hate coffee"]

print(tokenizer(sentences, padding="longest"))
print(tokenizer(sentences, padding="max_length"))

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint="distilbert-base-uncased-finetuned-sst-2-english"
tokenizer=AutoTokenizer.from_pretrained(checkpoint)
model=AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences=["I have been waiting for long", "I hate Coffee"]

tokens=tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")
output= model(**tokens)
print(output)

SequenceClassifierOutput(loss=None, logits=tensor([[ 2.2439, -1.9491],
        [ 3.9188, -3.2146]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
